In [1]:
import networkx as nx
from functools import partial
import pandas as pd
import numpy as np
import pickle
import _pickle as cPickle
#from networkit import linkprediction as lp, nxadapter
#lp for link-prediction, nxadapter for converting nx graph to networkit graph

import csv
import json
from networkx.readwrite import json_graph

from sklearn.metrics.pairwise import cosine_similarity
import tqdm

In [0]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

In [0]:
with open('./input/embeddings.pickle', 'rb') as handle:
  embeddings = pickle.load(handle)

In [0]:
valid_graph = nx.read_gpickle("./input/valid_graph.gpickle")
test_graph = nx.read_gpickle("./input/test_graph.gpickle")
train_graph = nx.read_gpickle("./input/train_graph.gpickle")

In [0]:
train.head()

,src,tar,label,cos_sim
0,0,2,0,-0.524723
1,0,10,0,0.335463
2,0,26,0,0.344200
3,0,43,0,-0.241197
4,0,57,0,-0.251312


In [0]:
test.head()

,src,tar,label
0,0,2,0
1,0,7,0
2,0,10,0
3,0,26,0
4,0,43,0


In [0]:
test['label'].value_counts()

0    1772223
1        578
Name: label, dtype: int64

In [0]:
train.label.value_counts()

0    1347041
1       1908
Name: label, dtype: int64

Calculating **Cosine Similarity** between source and target subreddit embeddings

In [0]:
for i in tqdm.tqdm(range(len(test))):
  test.loc[i, 'cos_sim'] = sum(np.array(list(embeddings[test_dict['src'][i]].values())) * np.array(list(embeddings[test_dict['tar'][i]].values())))

In [0]:
for i in tqdm.tqdm(range(len(train))):
  train.loc[i, 'cos_sim'] = sum(np.array(list(embeddings[train_dict['src'][i]].values())) * np.array(list(embeddings[train_dict['tar'][i]].values())))

Converting 'test' and 'train' to 'test_dict' and 'train_dict'

In [0]:
test_dict = test.to_dict(orient = 'series')
train_dict = train.to_dict(orient = 'series')

In [0]:
#Deductive Metric - DED

test_dict['DED'] = {}
for i in tqdm.tqdm(range(len(test_dict['src']))):
  try:
    test_dict['DED'][i] = len(set(test_graph.successors(test_dict['src'][i])).intersection(set(test_graph.predecessors(test_dict['tar'][i]))))/len(set(test_graph.successors(test_dict['src'][i])))
  except ZeroDivisionError:
    test_dict['DED'][i] = 0

100%|██████████| 1772801/1772801 [00:19<00:00, 89404.16it/s] 


In [0]:
train_dict['DED'] = {}
for i in tqdm.tqdm(range(len(train_dict['src']))):
  try:
    train_dict['DED'][i] = len(set(train_graph.successors(train_dict['src'][i])).intersection(set(train_graph.predecessors(train_dict['tar'][i]))))/len(set(train_graph.successors(train_dict['src'][i])))
  except ZeroDivisionError:
    train_dict['DED'][i] = 0

100%|██████████| 1348949/1348949 [02:00<00:00, 11158.00it/s]


In [0]:
#Inductive Metric - IND

test_dict['IND'] = {}
for i in tqdm.tqdm(range(len(test_dict['src']))):
  try:
    test_dict['IND'][i] = len(set(test_graph.predecessors(test_dict['src'][i])).intersection(set(test_graph.predecessors(test_dict['tar'][i]))))/len(set(test_graph.predecessors(test_dict['src'][i])))
  except ZeroDivisionError:
    test_dict['IND'][i] = 0

100%|██████████| 1772801/1772801 [00:16<00:00, 108703.48it/s]


In [0]:
train_dict['IND'] = {}
for i in tqdm.tqdm(range(len(train_dict['src']))):
  try:
    train_dict['IND'][i] = len(set(train_graph.predecessors(train_dict['src'][i])).intersection(set(train_graph.predecessors(train_dict['tar'][i]))))/len(set(train_graph.predecessors(train_dict['src'][i])))
  except ZeroDivisionError:
    train_dict['IND'][i] = 0

100%|██████████| 1348949/1348949 [01:54<00:00, 11770.83it/s]


In [0]:
#DED_LOG

test_dict['DED_LOG'] = {}
for i in tqdm.tqdm(range(len(test_dict['src']))):
  DED_SCORE = test_dict['DED'][i]
  if(DED_SCORE):
    test_dict['DED_LOG'][i] = DED_SCORE * np.log(len(set(test_graph.successors(test_dict['src'][i]))))
  else:
    test_dict['DED_LOG'][i] = 0

100%|██████████| 1772801/1772801 [00:11<00:00, 149520.09it/s]


In [0]:
train_dict['DED_LOG'] = {}
for i in tqdm.tqdm(range(len(train_dict['src']))):
  DED_SCORE = train_dict['DED'][i]
  if(DED_SCORE):
    train_dict['DED_LOG'][i] = DED_SCORE * np.log(len(set(train_graph.successors(train_dict['src'][i]))))
  else:
    train_dict['DED_LOG'][i] = 0

100%|██████████| 1348949/1348949 [00:46<00:00, 28752.98it/s]


In [0]:
#IND_LOG

test_dict['IND_LOG'] = {}
for i in tqdm.tqdm(range(len(test_dict['src']))):
  IND_SCORE = test_dict['IND'][i]
  if(IND_SCORE):
    test_dict['IND_LOG'][i] = IND_SCORE * np.log(len(set(test_graph.predecessors(test_dict['src'][i]))))
  else:
    test_dict['IND_LOG'][i] = 0

100%|██████████| 1772801/1772801 [00:06<00:00, 286986.70it/s]


In [0]:
train_dict['IND_LOG'] = {}
for i in tqdm.tqdm(range(len(train_dict['src']))):
  IND_SCORE = train_dict['IND'][i]
  if(IND_SCORE):
    train_dict['IND_LOG'][i] = IND_SCORE * np.log(len(set(train_graph.predecessors(train_dict['src'][i]))))
  else:
    train_dict['IND_LOG'][i] = 0

100%|██████████| 1348949/1348949 [00:22<00:00, 60412.14it/s] 


In [0]:
test_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA', 'SO', 'HPI', 'HDI', 'cos_sim'])

# Features for Directed Graphs

**New features based on direction of edges in graphs**

In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA', 'SO', 'HPI', 'HDI', 'cos_sim'])

In [0]:
# Feature 1 : Combined INF Score : DED + INF
test_dict['INF'] = dict(enumerate(np.add(list(test_dict['IND'].values()), list(test_dict['DED'].values()))))
train_dict['INF'] = dict(enumerate(np.add(list(train_dict['IND'].values()), list(train_dict['DED'].values()))))


# Feature 2 : Combined INF_2D Score : 2*DED + INF
test_dict['INF_2D'] = dict(enumerate(np.add(np.array(list(test_dict['IND'].values())), np.array(list(test_dict['DED'].values())) * 2)))
train_dict['INF_2D'] = dict(enumerate(np.add(np.array(list(train_dict['IND'].values())), np.array(list(train_dict['DED'].values())) * 2)))

# Feature 3 : Combined INF_2I Score : DED + 2*INF
test_dict['INF_2I'] = dict(enumerate(np.add(np.array(list(test_dict['IND'].values())) * 2, np.array(list(test_dict['DED'].values())))))
train_dict['INF_2I'] = dict(enumerate(np.add(np.array(list(train_dict['IND'].values())) * 2, np.array(list(train_dict['DED'].values())))))

# Feature 4 : Combined INF_LOG Score : DED_LOG + INF_LOG
test_dict['INF_LOG'] = dict(enumerate(np.add(np.array(list(test_dict['IND_LOG'].values())), np.array(list(test_dict['DED_LOG'].values())))))
train_dict['INF_LOG'] = dict(enumerate(np.add(np.array(list(train_dict['IND_LOG'].values())), np.array(list(train_dict['DED_LOG'].values())))))

# Feature 5 : Combined INF_LOG_2D Score : 2 *DED_LOG + INF_LOG
test_dict['INF_LOG_2D'] = dict(enumerate(np.add(np.array(list(test_dict['IND_LOG'].values())), np.array(list(test_dict['DED_LOG'].values())) * 2)))
train_dict['INF_LOG_2D'] = dict(enumerate(np.add(np.array(list(train_dict['IND_LOG'].values())), np.array(list(train_dict['DED_LOG'].values())) * 2)))

# Feature 6 : Combined INF_LOG_2I Score : DED_LOG + 2*INF_LOG
test_dict['INF_LOG_2I'] = dict(enumerate(np.add(np.array(list(test_dict['IND_LOG'].values())) * 2, np.array(list(test_dict['DED_LOG'].values())))))
train_dict['INF_LOG_2I'] = dict(enumerate(np.add(np.array(list(train_dict['IND_LOG'].values())) * 2, np.array(list(train_dict['DED_LOG'].values())))))

**Hierarchical Undirected Scores**

We reformulate classical link predictions features for undirected graph link prediction to directed graphs

In [0]:
undirected_test_graph = nx.to_undirected(test_graph)
undirected_train_graph = nx.to_undirected(train_graph)

In [0]:
# Feature 7: Common Neighbors
def common_neighbors(G, ebunch = None):

  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  return len(common_neighbors)


In [15]:
#test
test_dict['CN'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['CN'][i] = common_neighbors(undirected_test_graph, (src,tar))
  else:
    test_dict['CN'][i] = 0    
    
#train
train_dict['CN'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['CN'][i] = common_neighbors(undirected_train_graph,(src, tar))
  else:
    train_dict['CN'][i] = 0

100%|██████████| 1348949/1348949 [11:16<00:00, 1994.73it/s] 


In [11]:
test_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA', 'SO', 'HPI', 'HDI', 'cos_sim'])

In [0]:
# Feature 8: Resource Allocation Index

def resource_allocation_index(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    def predict(u, v):
        return sum(1 / G.degree(w) for w in nx.common_neighbors(G, u, v))
    return predict(ebunch[0], ebunch[1])

test_dict['RA'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['RA'][i] = resource_allocation_index(undirected_test_graph, (src, tar))
  else:
    test_dict['RA'][i] = 0    

train_dict['RA'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['RA'][i] = resource_allocation_index(undirected_train_graph, (src, tar))
  else:
    train_dict['RA'][i] = 0

100%|██████████| 1348949/1348949 [12:03<00:00, 1865.46it/s] 


In [0]:
test_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA'])

In [0]:
# Feature 9: Jaccard Coefficient

def jaccard_coefficient(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    def predict(u, v):
        cnbors = list(nx.common_neighbors(G, u, v))
        union_size = len(set(G[u]) | set(G[v]))
        if union_size == 0:
            return 0
        else:
            return len(cnbors) / union_size
    return predict(ebunch[0], ebunch[1])

test_dict['JC'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['JC'][i] = jaccard_coefficient(undirected_test_graph, (src, tar))
  else:
    test_dict['JC'][i] = 0

train_dict['JC'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['JC'][i] = jaccard_coefficient(undirected_train_graph, (src, tar))
  else:
    train_dict['JC'][i] = 0

100%|██████████| 1348949/1348949 [12:18<00:00, 1826.86it/s] 


In [0]:
test_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC'])

In [0]:
# Feature 10: Adamic Adar Index
import math

def adamic_adar_index(G, ebunch=None):
  if ebunch is None:
        ebunch = nx.non_edges(G)
  def predict(u, v):
      return sum(1 / math.log(G.degree(w))
                 for w in nx.common_neighbors(G, u, v))
  return predict(ebunch[0], ebunch[1])


test_dict['AA'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['AA'][i] = adamic_adar_index(undirected_test_graph, (src, tar))
  else:
    test_dict['AA'][i] = 0
    
train_dict['AA'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['AA'][i] = adamic_adar_index(undirected_train_graph, (src, tar))
  else:
    train_dict['AA'][i] = 0

100%|██████████| 1348949/1348949 [11:55<00:00, 1886.08it/s] 


In [0]:
test_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA'])

In [0]:
# Feature 11: Preferential Attachment

def preferential_attachment(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    return G.degree(ebunch[0]) * G.degree(ebunch[1])

test_dict['PA'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['PA'][i] = preferential_attachment(undirected_test_graph, (src,tar))
  else:
    test_dict['PA'][i] = 0    
    
train_dict['PA'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['PA'][i] = preferential_attachment(undirected_train_graph,(src, tar))
  else:
    train_dict['PA'][i] = 0

100%|██████████| 1348949/1348949 [00:21<00:00, 62980.31it/s]


In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA'])

In [0]:
#Feature 12: Resource Allocation based on Common Nighbour Interacions (RA-CNI)

def resource_allocation_CNI(G, ebunch = None):
  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  first = sum([1/(len(set(G.neighbors(i)))) for i in common_neighbors])
  
  neigbhor_x = G.neighbors(u)
  neigbhor_y = G.neighbors(v)
  
  second = sum([(1/(len(set(G.neighbors(cross[0]))))) - (1/(len(set(G.neighbors(cross[1]))))) for cross in list(itertools.product(list(neighbor_x), list(neighbor_y))) 
                if cross in list(undirected_test_graph.edges)])
  
  return second + first

In [0]:
test_dict['RA_CNI'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['RA_CNI'][i] = resource_allocation_CNI(undirected_test_graph, (src,tar))
  else:
    test_dict['RA_CNI'][i] = 0    
    
train_dict['RA_CNI'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['RA_CNI'][i] = resource_allocation_CNI(undirected_train_graph,(src, tar))
  else:
    train_dict['RA_CNI'][i] = 0

100%|██████████| 1348949/1348949 [17:16<00:00, 1301.27it/s] 


In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI'])

In [0]:
#Feature 13: The Salton Index (SA)

def salton_index(G, ebunch = None):
  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  return len(common_neighbors)/np.sqrt(len(set(G.neighbors(u)))*len(set(G.neighbors(v))))

In [0]:
test_dict['SA'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['SA'][i] = salton_index(undirected_test_graph, (src,tar))
  else:
    test_dict['SA'][i] = 0    
    
train_dict['SA'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['SA'][i] = salton_index(undirected_train_graph,(src, tar))
  else:
    train_dict['SA'][i] = 0

100%|██████████| 1348949/1348949 [11:50<00:00, 1898.31it/s] 


In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA'])

In [0]:
#Feature 14: The Sørensen Index (SO)

def sorenson_index(G, ebunch = None):
  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  return (2 * len(common_neighbors))/(len(set(G.neighbors(u))) + len(set(G.neighbors(v))))

In [0]:
test_dict['SO'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['SO'][i] = sorenson_index(undirected_test_graph, (src,tar))
  else:
    test_dict['SO'][i] = 0    
    
train_dict['SO'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['SO'][i] = sorenson_index(undirected_train_graph,(src, tar))
  else:
    train_dict['SO'][i] = 0

100%|██████████| 1348949/1348949 [11:45<00:00, 1912.03it/s] 


In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA', 'SO'])

In [0]:
#Feature 15: Hub Promoted Index (HPI)

def hub_promoted_index(G, ebunch = None):
  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  return len(common_neighbors)/min(len(set(G.neighbors(u))) , len(set(G.neighbors(v))))

In [0]:
test_dict['HPI'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['HPI'][i] = hub_promoted_index(undirected_test_graph, (src,tar))
  else:
    test_dict['HPI'][i] = 0    
    
train_dict['HPI'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['HPI'][i] = hub_promoted_index(undirected_train_graph,(src, tar))
  else:
    train_dict['HPI'][i] = 0

100%|██████████| 1348949/1348949 [11:45<00:00, 1910.91it/s] 


In [0]:
train_dict.keys()

dict_keys(['src', 'tar', 'label', 'DED', 'IND', 'DED_LOG', 'IND_LOG', 'INF', 'INF_2D', 'INF_2I', 'INF_LOG', 'INF_LOG_2D', 'INF_LOG_2I', 'RA', 'JC', 'AA', 'PA', 'RA_CNI', 'SA', 'SO', 'HPI'])

In [0]:
#Feature 16: Hub Depressed Index (HDI)

def hub_depressed_index(G, ebunch = None):
  u = ebunch[0]
  v = ebunch[1]
  
  common_neighbors =  [w for w in G[u] if w in G[v] and w not in (u, v)]
  return len(common_neighbors)/max(len(set(G.neighbors(u))) , len(set(G.neighbors(v))))

In [0]:
#test
test_dict['HDI'] = {}

for i in tqdm.tqdm(range(len(test_dict['src']))):
  
  src = test_dict['src'][i]
  tar = test_dict['tar'][i]
  
  if(test_graph.in_degree(src) > test_graph.in_degree(tar)):
    test_dict['HDI'][i] = hub_depressed_index(undirected_test_graph, (src,tar))
  else:
    test_dict['HDI'][i] = 0    
    

#train
train_dict['HDI'] = {}

for i in tqdm.tqdm(range(len(train_dict['src']))):
  
  src = train_dict['src'][i]
  tar = train_dict['tar'][i]
  
  if(train_graph.in_degree(src) > train_graph.in_degree(tar)):
    train_dict['HDI'][i] = hub_depressed_index(undirected_train_graph,(src, tar))
  else:
    train_dict['HDI'][i] = 0

100%|██████████| 1348949/1348949 [11:49<00:00, 1902.57it/s] 


In [0]:
test_dict_df = pd.DataFrame(test_dict)
train_dict_df = pd.DataFrame(train_dict)

In [0]:
test_dict_df.to_csv('./input/test_dict_df.csv', header = True, index = False)
train_dict_df.to_csv('./input/train_dict_df.csv', header = True, index = False)